In [ ]:
import gymnasium as gym
import numpy as np

from stable_baselines3 import DQN
from stable_baselines3.common.noise import NormalActionNoise

from src.models.Qlearning import QAgent
from src.env.TestEnv import HydroElectric_Test
import matplotlib.pyplot as plt
import numpy as np
import random

In [11]:
seed = 5
np.random.seed(seed)
random.seed(seed)

In [12]:
env = HydroElectric_Test("train.xlsx")
env.reset(seed=seed)
# env.discrete_action_space.seed(seed)
#env_wrapped = StepWrapper(env, step_hours=1)
agent = QAgent(env)

# step_hours = 3 
# agent = QAgent(env)

In [8]:
type(env.discrete_action_space)

gymnasium.spaces.discrete.Discrete

In [13]:
model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000, log_interval=4)


obs, info = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        obs, info = env.reset()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


AttributeError: 'HydroElectric_Test' object has no attribute 'observation_space'